# setup

In [3]:
import sys
sys.path.append('../../..')

In [4]:
from copulae.input import generate_copula_net_input
from copulae.training.cflax.mixtures import DoubleIntegral
from copulae.training.cflax.mixtures import MixtureCopula
from copulae.training.cflax.mixtures import FrankCopNet

from copulae.training.loss import sq_error, sq_error_partial, copula_likelihood
from copulae.training import setup_training

In [5]:
def build_mixture(base_class, n, *args, **kwargs):
    rv = []
    for i in range(n):
        rv.append(base_class(*args, **kwargs))
    return MixtureCopula(rv)

In [6]:
import jax.numpy as jnp
import jax.scipy.stats as jss
import jax

from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import optax
import pickle
from scipy.stats import bootstrap

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# data
https://github.com/yutingng/gen-AC.git

In [8]:
import pandas as pd
import scipy
import copy

from sklearn.model_selection import train_test_split

In [9]:
def add_train_random_noise(data, num_adds):
  new_data = np.random.rand(num_adds, data.shape[1])
  return np.concatenate((data, new_data), axis = 0)

In [10]:
def rank_normalization(X):
  X = copy.deepcopy(X)
  for z in X:
      ndata = z.shape[0]
      gap = 1./(ndata+1)
      nfeats = z.shape[1]
      for i in range(nfeats):
          z[:, i] = scipy.stats.rankdata(z[:, i], 'ordinal')*gap
  return X

In [11]:
!git clone https://github.com/yutingng/gen-AC.git

Cloning into 'gen-AC'...
remote: Enumerating objects: 466, done.
remote: Counting objects: 100% (466/466), done.
remote: Compressing objects: 100% (339/339), done.
remote: Total 466 (delta 159), reused 421 (delta 123), pack-reused 0
Receiving objects: 100% (466/466), 10.28 MiB | 7.09 MiB/s, done.
Resolving deltas: 100% (159/159), done.


In [12]:
class Boston():
  def __init__(self):
    # read
    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep = "\s+", skiprows = 22, header = None)
    X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
    y = raw_df.values[1::2, 2]

    # split
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, random_state = 142857)
    X_train = np.concatenate((X_train, y_train[:, None]), axis = 1)
    X_test  = np.concatenate((X_test, y_test[:, None]), axis = 1)

    # norm
    [X_train, X_test] = rank_normalization([X_train, X_test])

    # noise
    X_train = add_train_random_noise(X_train, int(X_train.shape[0]*0.01))

    # 2d
    train_data = X_train[:, [0, 13]]
    test_data = X_test[:, [0, 13]]

    # flip
    train_data[:, 0] = 1 - train_data[:, 0]
    test_data[:, 0] = 1 - test_data[:, 0]

    self.train_y = train_data[:, 1].reshape(-1, 1)
    self.train_x = train_data[:, 0].reshape(-1, 1)
    self.validation_y = test_data[:, 1].reshape(-1, 1)
    self.validation_x = test_data[:, 0].reshape(-1, 1)

In [13]:
class INTC_MSFT():
  def __init__(self):
    # read
    intel_f = open('gen-AC/data/raw/INTC_MSFT_GE/INTEL.data', 'r')
    intel = np.array(list(map(float, intel_f.readlines())))

    ms_f = open('gen-AC/data/raw/INTC_MSFT_GE/MS.data', 'r')
    ms = np.array(list(map(float, ms_f.readlines())))

    ge_f = open('gen-AC/data/raw/INTC_MSFT_GE/GE.data', 'r')
    ge = np.array(list(map(float, ge_f.readlines())))

    # split
    X = np.concatenate((intel[:, None], ms[:, None]), axis = 1)
    X_train, X_test, _, _ = train_test_split(X, X, shuffle = True, random_state = 142857)

    # norm
    [X_train, X_test] = rank_normalization([X_train, X_test])

    # 2d, noise
    train_data = X_train[:, [0, 1]]
    train_data = add_train_random_noise(train_data, int(train_data.shape[0]*0.01))
    test_data = X_test[:, [0, 1]]

    self.train_y = train_data[:, 1].reshape(-1, 1)
    self.train_x = train_data[:, 0].reshape(-1, 1)
    self.validation_y = test_data[:, 1].reshape(-1, 1)
    self.validation_x = test_data[:, 0].reshape(-1, 1)

In [14]:
class GOOG_FB():
  def __init__(self):
    # read
    goog_f = open('gen-AC/data/raw/FB_GOOG/goog/close.vals', 'r')
    goog = np.array(list(map(float, goog_f.readlines())))

    fb_f = open('gen-AC/data/raw/FB_GOOG/fb/close.vals', 'r')
    fb = np.array(list(map(float, fb_f.readlines())))

    # split
    X = np.concatenate((goog[:, None], fb[:, None]), axis = 1)
    X_train, X_test, _, _ = train_test_split(X, X, shuffle=True, random_state=142857)

    # norm
    [X_train, X_test] = rank_normalization([X_train, X_test])

    # 2d, noise
    train_data = X_train[:, [0, 1]]
    train_data = add_train_random_noise(train_data, int(train_data.shape[0]*0.01))
    test_data = X_test[:, [0, 1]]

    self.train_y = train_data[:, 1].reshape(-1, 1)
    self.train_x = train_data[:, 0].reshape(-1, 1)
    self.validation_y = test_data[:, 1].reshape(-1, 1)
    self.validation_x = test_data[:, 0].reshape(-1, 1)

In [15]:
def get_set(D_val, data_points):
  points = D_val
  points = jnp.expand_dims(points, axis=0)

  # PDF and CDF for X
  kde_x = jss.gaussian_kde(data_points[0], bw_method='silverman')
  density_x = kde_x.evaluate(points[0, 0, :])
  cumulative_x = jnp.array([kde_x.integrate_box_1d(-jnp.inf, p) for p in points[0, 0, :]])

  # PDF and CDF for Y
  kde_y = jss.gaussian_kde(D[1], bw_method='silverman')
  density_y = kde_y.evaluate(points[0, 1, :])
  cumulative_y = jnp.array([kde_y.integrate_box_1d(-jnp.inf, p) for p in points[0, 1, :]])

  I_pdf = density_x.T * density_y.T
  I_pdf = jnp.expand_dims(I_pdf, axis=0)
  cdf_xy = jnp.array((cumulative_x, cumulative_y))
  cdf_xy = jnp.expand_dims(cdf_xy, axis=0)

  del density_x
  del density_y
  del cumulative_x
  del cumulative_y

  return points, I_pdf, cdf_xy

In [16]:
np.random.seed(30091985)
key = jax.random.PRNGKey(30091985)

# params

In [17]:
layer_widths = [128, 64, 32, 16]
losses = [
    (0.01, sq_error),
    (0.5, sq_error_partial),
    (0.1, copula_likelihood),
]
lr = 2e-3
n_iter = 10000
label = '2catspfr10'

In [18]:
def L_d(losses, params, state):
  loss = jnp.zeros((1,len(losses)), dtype=jnp.float32)
  for i, (w, loss_func) in enumerate(losses):
    loss = loss.at[0, i].set(w * loss_func(params, state))
  return loss

losses_eval = [
    (1.0, sq_error),
    (1.0, sq_error_partial),
    (1.0, copula_likelihood),
]

# boston

In [19]:
model = DoubleIntegral(
    build_mixture(FrankCopNet, 10)
)

In [20]:
data_loader = Boston()
D = np.array([data_loader.train_x, data_loader.train_y])[:, :, 0]
TrainingTensors = generate_copula_net_input(
    D=D,
    bootstrap=False
)

In [21]:
nn_C, nn_dC, nn_c, cop_state, forward, grad = setup_training(
    model, TrainingTensors, losses
)

In [22]:
key, subkey = jax.random.split(key)
init_params = model.init(subkey, TrainingTensors.UV_batches[0])
del subkey

params = init_params
optimizer = optax.adam(lr)
opt_state = optimizer.init(params)

In [23]:
best = 1e6
for i in tqdm(range(n_iter)):
    grads, cop_state = grad(params, cop_state)
    updates, opt_state = optimizer.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    if i % 10 == 0:
        loss = L_d(losses_eval, params, cop_state)
        if loss[0][-1] < best:
          best_params = params
          best_cop_state = cop_state
          best = loss[0][-1]
        print('Iter {}. Loss {}'.format(i, loss))

  0%|          | 1/10000 [03:02<507:02:01, 182.55s/it]

Iter 0. Loss [[0.10170246 0.19985475 2.8594708 ]]


  0%|          | 12/10000 [06:08<8:06:23,  2.92s/it] 

Iter 10. Loss [[0.10143276 0.19862682 2.8205183 ]]


  0%|          | 21/10000 [06:10<46:01,  3.61it/s]

...

100%|█████████▉| 9981/10000 [44:38<00:05,  3.39it/s]

Iter 9980. Loss [[0.13556616 0.10873841 1.5520216 ]]


100%|█████████▉| 9992/10000 [44:41<00:01,  4.66it/s]

Iter 9990. Loss [[0.13556617 0.10873841 1.5520217 ]]


100%|██████████| 10000/10000 [44:43<00:00,  3.73it/s]


In [24]:
density_graph_points, I_pdf, cdf_xy = get_set(D, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))
np.mean(yhat)

1.1886039

In [25]:
D_val = np.array([data_loader.validation_x, data_loader.validation_y])[:, :, 0]
density_graph_points, I_pdf, cdf_xy = get_set(D_val, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))

L_d(losses_eval, best_params, best_cop_state), np.mean(yhat)

(Array([[0.13556443, 0.10873897, 1.5520208 ]], dtype=float32), 1.1256244)

In [26]:
res = bootstrap(yhat, np.mean)
res.standard_error, res.confidence_interval

(0.057455394,
 ConfidenceInterval(low=1.022850129935594, high=1.2509258252335393))

In [27]:
with open('drive/MyDrive/copulae/models/{}_boston.pkl'.format(label), 'wb') as fp:
    pickle.dump(best_params, fp)

# intc-msft

In [28]:
model = DoubleIntegral(
    build_mixture(FrankCopNet, 10)
)

In [29]:
data_loader = INTC_MSFT()
D = np.array([data_loader.train_x, data_loader.train_y])[:, :, 0]
TrainingTensors = generate_copula_net_input(
    D=D,
    bootstrap=False
)

In [30]:
nn_C, nn_dC, nn_c, cop_state, forward, grad = setup_training(
    model, TrainingTensors, losses
)

In [31]:
key, subkey = jax.random.split(key)
init_params = model.init(subkey, TrainingTensors.UV_batches[0])
del subkey

params = init_params
optimizer = optax.adam(lr)
opt_state = optimizer.init(params)

In [32]:
best = 1e6
for i in tqdm(range(n_iter)):
    grads, cop_state = grad(params, cop_state)
    updates, opt_state = optimizer.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    if i % 10 == 0:
        loss = L_d(losses_eval, params, cop_state)
        if loss[0][-1] < best:
          best_params = params
          best_cop_state = cop_state
          best = loss[0][-1]
        print('Iter {}. Loss {}'.format(i, loss))

  0%|          | 1/10000 [03:01<504:36:17, 181.68s/it]

Iter 0. Loss [[0.1276773 0.2152244 2.7093182]]


  0%|          | 12/10000 [06:03<8:00:26,  2.89s/it] 

Iter 10. Loss [[0.12639403 0.21279746 2.664716  ]]


  0%|          | 22/10000 [06:05<41:39,  3.99it/s]

...

100%|█████████▉| 9982/10000 [46:43<00:03,  4.64it/s]

Iter 9980. Loss [[0.17152275 0.08659006 1.0976833 ]]


100%|█████████▉| 9992/10000 [46:44<00:01,  5.34it/s]

Iter 9990. Loss [[0.17152067 0.0865911  1.0976784 ]]


100%|██████████| 10000/10000 [46:46<00:00,  3.56it/s]


In [33]:
density_graph_points, I_pdf, cdf_xy = get_set(D, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))
np.mean(yhat)

0.8707999

In [34]:
D_val = np.array([data_loader.validation_x, data_loader.validation_y])[:, :, 0]
density_graph_points, I_pdf, cdf_xy = get_set(D_val, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))

L_d(losses_eval, best_params, best_cop_state), np.mean(yhat)

(Array([[0.17149976, 0.08660112, 1.0976346 ]], dtype=float32), 0.889027)

In [35]:
res = bootstrap(yhat, np.mean)
res.standard_error, res.confidence_interval

(0.03499488,
 ConfidenceInterval(low=0.8273708723464309, high=0.9675899605254292))

In [36]:
with open('drive/MyDrive/copulae/models/{}_intcmsft.pkl'.format(label), 'wb') as fp:
    pickle.dump(best_params, fp)

# goog-fb

In [37]:
model = DoubleIntegral(
    build_mixture(FrankCopNet, 10)
)

In [38]:
data_loader = GOOG_FB()
D = np.array([data_loader.train_x, data_loader.train_y])[:, :, 0]
TrainingTensors = generate_copula_net_input(
    D=D,
    bootstrap=False
)

In [39]:
nn_C, nn_dC, nn_c, cop_state, forward, grad = setup_training(
    model, TrainingTensors, losses
)

In [40]:
key, subkey = jax.random.split(key)
init_params = model.init(subkey, TrainingTensors.UV_batches[0])
del subkey

params = init_params
optimizer = optax.adam(lr)
opt_state = optimizer.init(params)

In [41]:
best = 1e6
for i in tqdm(range(n_iter)):
    grads, cop_state = grad(params, cop_state)
    updates, opt_state = optimizer.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    if i % 10 == 0:
        loss = L_d(losses_eval, params, cop_state)
        if loss[0][-1] < best:
          best_params = params
          best_cop_state = cop_state
          best = loss[0][-1]
        print('Iter {}. Loss {}'.format(i, loss))

  0%|          | 1/10000 [03:05<514:31:09, 185.25s/it]

Iter 0. Loss [[0.16888434 0.21504277 2.3328278 ]]


  0%|          | 12/10000 [06:00<7:43:48,  2.79s/it] 

Iter 10. Loss [[0.1679515 0.2132664 2.298576 ]]


  0%|          | 21/10000 [06:03<1:10:00,  2.38it/s]

...

100%|█████████▉| 9982/10000 [47:18<00:03,  5.46it/s]

Iter 9980. Loss [[0.24968316 0.10732029 0.3274819 ]]


100%|█████████▉| 9991/10000 [47:21<00:02,  3.07it/s]

Iter 9990. Loss [[0.24968201 0.10732032 0.32748175]]


100%|██████████| 10000/10000 [47:23<00:00,  3.52it/s]


In [42]:
density_graph_points, I_pdf, cdf_xy = get_set(D, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))
np.mean(yhat)

0.35669515

In [43]:
D_val = np.array([data_loader.validation_x, data_loader.validation_y])[:, :, 0]
density_graph_points, I_pdf, cdf_xy = get_set(D_val, best_cop_state.X_batches[0])

copula_density = nn_c(best_params, cdf_xy)
points_density = copula_density * I_pdf
yhat = -np.log(jnp.clip(points_density, 1e-6))

L_d(losses_eval, best_params, best_cop_state), np.mean(yhat)

(Array([[0.24967511, 0.10732554, 0.3274637 ]], dtype=float32), 0.3545602)

In [44]:
res = bootstrap(yhat, np.mean)
res.standard_error, res.confidence_interval

(0.025551734,
 ConfidenceInterval(low=0.30716392133325776, high=0.4071947492205935))

In [45]:
with open('drive/MyDrive/copulae/models/{}_googfb.pkl'.format(label), 'wb') as fp:
    pickle.dump(best_params, fp)